<a href="https://colab.research.google.com/github/jonathanluo23/NBA-allstar-all-nba-Prediction/blob/main/Feature_Selection_All_NBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the necessary libraries first
#### Correlation method - for classification chi2 only
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#### RFE method - replace the model with the model you plan to use
from sklearn.feature_selection import RFE
# from sklearn.linear_model import LogisticRegression
#### feature importance method
#### this method can be used for both topK and cut-off
from sklearn.linear_model import Ridge
#### specifically tree-based feature importance method
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
#### for voting
from collections import Counter

import pandas as pd
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Capstone/Complete_final_allnba')

In [ ]:
data

,GP_x,W,L,W_PCT_x,OFF_RATING,DEF_RATING,NET_RATING,AST_PCT,AST_TO,TM_TOV_PCT,...,PLUS_MINUS,label_year_2012-13,label_year_2014-15,label_year_2016-17,label_year_2018-19,label_year_2020-22,label_position_C or C-F,label_position_F or F-G or F-C,label_position_G or G-F,Y
0,0.516245,0.512548,0.344406,0.605362,0.337814,0.216889,0.568634,0.733510,0.734789,0.501950,...,0.588911,1,0,0,0,0,0,0,1,0
1,0.577773,0.371560,0.555366,0.331044,0.267100,0.514761,0.282726,0.256045,0.220034,0.988317,...,0.299963,1,0,0,0,0,1,0,0,0
2,0.763312,0.623642,0.502553,0.551767,0.588992,0.378389,0.673786,0.356908,0.305738,0.514080,...,0.718178,1,0,0,0,0,0,1,0,0
3,0.116458,0.195063,0.108972,0.628648,0.621990,0.304073,0.766851,0.448047,0.496716,0.501950,...,0.867368,1,0,0,0,0,1,0,0,0
4,0.726100,0.575695,0.515948,0.512191,0.628011,0.456607,0.648635,0.448047,0.637905,0.102801,...,0.718178,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5622,0.800570,0.635318,0.529211,0.540279,0.740548,0.753699,0.512625,0.651614,0.557765,0.439831,...,0.501347,0,0,0,0,1,0,0,1,0
5623,0.034345,0.039722,0.085600,0.235418,0.329273,1.000000,0.000000,0.715559,0.786310,0.514080,...,0.000000,0,0,0,0,1,0,0,1,0
5624,0.479417,0.472608,0.344406,0.574820,0.580027,0.649248,0.453374,0.181626,0.290082,0.414245,...,0.449558,0,0,0,0,1,0,1,0,0
5625,0.738499,0.680907,0.404546,0.655231,0.752845,0.593894,0.651424,0.282863,0.517638,0.307031,...,0.675272,0,0,0,0,1,0,1,0,0


In [ ]:
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [ ]:
names = list(data.iloc[:,:-1])
names

['GP_x',
 'W',
 'L',
 'W_PCT_x',
 'OFF_RATING',
 'DEF_RATING',
 'NET_RATING',
 'AST_PCT',
 'AST_TO',
 'TM_TOV_PCT',
 'TS_PCT',
 'OREB_PCT',
 'DREB_PCT',
 'REB_PCT',
 'EFG_PCT',
 'PIE',
 'USG_PCT',
 'TS_PCT.1',
 'PACE',
 'POSS',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'PF',
 'PTS',
 'PLUS_MINUS',
 'label_year_2012-13',
 'label_year_2014-15',
 'label_year_2016-17',
 'label_year_2018-19',
 'label_year_2020-22',
 'label_position_C or C-F',
 'label_position_F or F-G or F-C',
 'label_position_G or G-F']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, y,test_size=.25,random_state=2021)

In [ ]:
def feature_selector(X, y, model, names, _method="topk", n=int(X.shape[1]/2), fit_X=False, thres = 0.1):
  """voting based feature selector
  - _method: "topk" for top-K method (default), "cutoff" for cut-off based method
  - n: number of features to be selected. only available for top-K method - default half of the total features
  - thres: cut-off threshold (default 0.1), only availabel for cutoff method
  - fit_X: fit_transform X or just return indices
  TODO: include support for regression problems
  """
  #### Placeholder for tests
  # X.shape[0] == y.shape[0]
  if names: # if given feature names
    feature_names = np.array(names)
  else: # otherwise use location
    feature_names = np.array(["X%s" % x for x in range(len(X.shape[0]))])
  if _method == 'topk':
    #### np.argpartition gets the indices of the largest n element from the array in ascending order
    #### [::-1] reverse the order
    corr_features = list(np.argpartition(SelectKBest(score_func=chi2, k=n).fit(X,y).scores_, -n)[-n:][::-1])
    #### binary masking on features, use np.where() to get the indices of selected
    rfe_features = list(np.where(RFE(model, n_features_to_select=n, step=1).fit(X, y).support_)[0])
    #### argsort get sorted indices by values, ::-1] reverse the asceding order
    ridge_features = list(np.argsort(Ridge(alpha=1.0).fit(X, y).coef_)[-n:][::-1])
    #### using ExtraTree
    extratree_features = list(np.argsort(ExtraTreesClassifier().fit(X, y).feature_importances_)[-n:][::-1])
    ### using Random Forest:
    rf_features = list(np.argsort(RandomForestClassifier().fit(X, y).feature_importances_)[-n:][::-1])
    print("Using the Top-K method: ")
    print("Selected features by correlation: ", feature_names[corr_features])
    print("Selected features by RFE: ", feature_names[rfe_features])
    print("Selected features by Ridge coefficients: ", feature_names[ridge_features])
    print("Selected features by Extra Tree feature importance: ", feature_names[extratree_features])
    print("Selected features by Random Forest feature importance: ", feature_names[rf_features])

  elif _method == 'cutoff':
    corr_features, rfe_features = list(), list() #### N/A
    ridge_features = list(np.where(Ridge(alpha=1.0).fit(X, y).coef_ > thres)[0])
    extratree_features = list(np.where(ExtraTreesClassifier().fit(X, y).feature_importances_ > thres)[0])
    rf_features = list(np.where(RandomForestClassifier().fit(X, y).feature_importances_ > thres)[0])
    print("Using the Cutoff method: ")
    # print("Selected features by correlation: ", feature_names[corr_features])
    # print("Selected features by RFE: ", feature_names[rfe_features])
    print("Selected features by Ridge coefficients: ", feature_names[ridge_features])
    print("Selected features by Extra Tree feature importance: ", feature_names[extratree_features])
    print("Selected features by Random Forest feature importance: ", feature_names[rf_features])
  else:
    return("Only Top-K and Cutoff methods are currently supported!")

  ######################################
  #### combine results using voting ####
  ######################################
  counted = Counter(np.concatenate((corr_features, rfe_features, ridge_features, extratree_features, rf_features), axis=None)).most_common(n)

  #### list of tuples (feature_index, votes)
  counted.sort(key = lambda x: x[1], reverse=True)
  final_select_series = pd.Series({feature_names[f]:c for f,c in counted}).sort_values(ascending=False)
  selected_idx = sorted([f for f,c in counted])
  assert len(selected_idx) == n
  print(final_select_series)
  if fit_X:
    return(X[:,selected_idx])
  else:
    return(feature_names[selected_idx])

In [ ]:
selected_features = feature_selector(X_train, y_train, model=RandomForestClassifier(), n=10, names=names, fit_X=False)
selected_features

Using the Top-K method: 
Selected features by correlation:  ['FTA' 'FTM' 'FGM' 'PIE' 'PTS' 'STL' 'AST' 'TOV' 'PLUS_MINUS' 'POSS']
Selected features by RFE:  ['W' 'W_PCT_x' 'PIE' 'POSS' 'FGM' 'FGA' 'FTM' 'FTA' 'PTS' 'PLUS_MINUS']
Selected features by Ridge coefficients:  ['PTS' 'PLUS_MINUS' 'POSS' 'FTA' 'PIE' 'FG3A' 'AST_PCT' 'GP_x' 'TOV'
 'DREB']
Selected features by Extra Tree feature importance:  ['PIE' 'FTA' 'PTS' 'FTM' 'PLUS_MINUS' 'FGM' 'USG_PCT' 'FGA' 'TOV' 'POSS']
Selected features by Random Forest feature importance:  ['PTS' 'FGM' 'PIE' 'FTA' 'W' 'PLUS_MINUS' 'FGA' 'POSS' 'FTM' 'USG_PCT']
FTA           5
PIE           5
PTS           5
PLUS_MINUS    5
POSS          5
FTM           4
FGM           4
TOV           3
FGA           3
W             2
dtype: int64


array(['W', 'PIE', 'POSS', 'FGM', 'FGA', 'FTM', 'FTA', 'TOV', 'PTS',
       'PLUS_MINUS'], dtype='<U30')

In [ ]:
features = data[['PTS', 'FGM', 'PIE', 'FTA', 'W', 'PLUS_MINUS', 'FGA', 'POSS', 'FTM', 'USG_PCT','Y']]

In [ ]:
features.corr()

,PTS,FGM,PIE,FTA,W,PLUS_MINUS,FGA,POSS,FTM,USG_PCT,Y
PTS,1.000000,0.991464,0.698846,0.863498,0.540210,0.256753,0.978051,0.828209,0.885678,0.653396,0.329857
FGM,0.991464,1.000000,0.708214,0.839365,0.538994,0.250413,0.973822,0.826661,0.853801,0.646992,0.319505
PIE,0.698846,0.708214,1.000000,0.660124,0.413285,0.360988,0.606837,0.525249,0.667568,0.529265,0.355386
FTA,0.863498,0.839365,0.660124,1.000000,0.441379,0.202011,0.813709,0.707510,0.979700,0.603736,0.333196
W,0.540210,0.538994,0.413285,0.441379,1.000000,0.507304,0.500941,0.805269,0.455126,0.160421,0.219263
PLUS_MINUS,0.256753,0.250413,0.360988,0.202011,0.507304,1.000000,0.202368,0.267296,0.210933,0.106811,0.299880
FGA,0.978051,0.973822,0.606837,0.813709,0.500941,0.202368,1.000000,0.810653,0.839255,0.683400,0.301482
POSS,0.828209,0.826661,0.525249,0.707510,0.805269,0.267296,0.810653,1.000000,0.723174,0.332551,0.234640
FTM,0.885678,0.853801,0.667568,0.979700,0.455126,0.210933,0.839255,0.723174,1.000000,0.623938,0.326712
USG_PCT,0.653396,0.646992,0.529265,0.603736,0.160421,0.106811,0.683400,0.332551,0.623938,1.000000,0.307172


In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
features.to_csv('/content/drive/MyDrive/Capstone/features_allnba',index=False)